## Carga de los Datasets
Cargamos en el notebook los ficheros de cardio_train.csv y train_2v.csv, ambos datasets se utilizan para evaluar si una persona sufrió un paro cardiaco o no 

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_70edc0d029d3420b8a46ea6c4e12c9fd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9VbZC-P3eJQ0LvTR4-8uXkoJq5sknHRg-YxdgMVidL5I',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_70edc0d029d3420b8a46ea6c4e12c9fd.get_object(Bucket='byebypass-donotdelete-pr-0lgcus86xella8',Key='cardio_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

cardio_data = pd.read_csv(body)


ModuleNotFoundError: No module named 'botocore'

In [ ]:
body = client_70edc0d029d3420b8a46ea6c4e12c9fd.get_object(Bucket='byebypass-donotdelete-pr-0lgcus86xella8',Key='train_2v.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_data = pd.read_csv(body)
train_data.head()

In [61]:
import pandas as pd
cardio_data = pd.read_csv('cardio_train.csv', sep=';')
train_data = pd.read_csv('train_2v.csv')


### Cardio CSV 
_Variables del Dataset_
- **Age**:  
- **Gender** 
- **Height** 
- **Weightr**
- **Ap_hi**
- **Ap_lo** 
- **Choleterol**
- **Gluc** 
- **Smoke**
- **Alco** 
- **Active** 
- **Cardio** 

In [62]:
cardio_data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


### Train 2v CSV 
_Variables del Dataset_

- **Gender** 
- **Age** 
- **Hypertension** 
- **Heart Disease** 
- **Ever Married** 
- **Work Type** 
- **Residence Type** 
- **AVG_Glucose_Level** 
- **BMI** 
- **Smoking_Status** 
- **Stroke**  




In [63]:
train_data.head() 

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


#### Dado que algunas variables de un dataset, no se consideran en el otro, se eliminarán de ambos datasets, estas columnas:
- cardio_data.csv: alco, active
- train_data.csv: ever_married, work_type, residence_type

In [64]:
cardio_data=cardio_data.drop(['alco', 'active'], axis=1) 
cardio_data.head() 

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0
1,1,20228,1,156,85.0,140,90,3,1,0,1
2,2,18857,1,165,64.0,130,70,3,1,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,1
4,4,17474,1,156,56.0,100,60,1,1,0,0


In [65]:
train_data=train_data.drop(['ever_married', 'work_type','Residence_type'], axis=1) 
train_data.head() 

,id,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,161.28,19.1,NaN,0


#### A continuación empezaremos con las transformaciones de algunas columnas.
Esto se hace debido a que se observa que una o varias variables de un dataset está o están directamente relacionadas con el del otro. También se harán en los casos en los que la misma variable en ambos datasets representen de distinta forma un mismo valor.

#### 1) "age"  days  (cardio_data) <=> "age"  days  (cardio_data)

In [66]:
cardio_data["age"]=round(cardio_data["age"]/365, 0)

#### 2) "weight" y "height" (cardio_data) <=> "bmi" (train_data)

In [67]:
cardio_data["bmi"]=round (cardio_data["weight"] / ((cardio_data["height"]/100)**2), 0)


In [68]:
cardio_data=cardio_data.drop(['height', 'weight'], axis=1) 
cardio_data.head() 

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi
0,0,50.0,2,110,80,1,1,0,0,22.0
1,1,55.0,1,140,90,3,1,0,1,35.0
2,2,52.0,1,130,70,3,1,0,1,24.0
3,3,48.0,2,150,100,1,1,0,1,29.0
4,4,48.0,1,100,60,1,1,0,0,23.0


#### 3) "gender" (cardio_data) => Modificamos el valor 1 a 1 (mujer) y 2 a 0 (hombre) para que quede como en train_data

In [69]:
cardio_data["gender"]=cardio_data["gender"].map({1:1,2:0}) 

#### 4) Modificamos "ap_hi" y "ap_lo" (cardio_data) para definir si se tiene la presión alta o no: "high_tension" (tensión alta (>= 80 / 140) = 1, tensión normal o baja = 0)

#### Definimos también una nueva columna "hypertension" (cardio_data) con los valores 0 (no sufre hipertensión) y 1 (sí la sufre) para relacionarla con "hypertension" (train_data)

In [70]:
#Valores hipertensión: 140-90

cardio_data[(cardio_data["ap_hi"]>=140) & (cardio_data["ap_lo"]>=80)]


,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi
1,1,55.0,1,140,90,3,1,0,1,35.0
3,3,48.0,0,150,100,1,1,0,1,29.0
22,32,63.0,1,145,85,2,2,0,1,36.0
24,35,46.0,1,150,90,3,1,0,1,24.0
31,43,64.0,1,140,90,3,1,0,1,25.0
34,46,60.0,0,140,90,3,1,0,1,27.0
39,53,50.0,1,140,90,1,1,0,1,26.0
42,57,59.0,0,140,90,1,1,0,1,24.0
44,59,54.0,1,170,80,3,1,0,1,21.0
45,60,48.0,1,140,90,1,1,0,1,24.0


In [71]:
cardio_data["high_tension"]=(cardio_data["ap_hi"]>=140) & (cardio_data["ap_lo"]>=80)
cardio_data.head()

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi,high_tension
0,0,50.0,0,110,80,1,1,0,0,22.0,False
1,1,55.0,1,140,90,3,1,0,1,35.0,True
2,2,52.0,1,130,70,3,1,0,1,24.0,False
3,3,48.0,0,150,100,1,1,0,1,29.0,True
4,4,48.0,1,100,60,1,1,0,0,23.0,False


In [73]:
cardio_data[(cardio_data["high_tension"]==True)]
cardio_data["high_tension"]=cardio_data["high_tension"].map({True:1,False:0}) 
cardio_data.head()

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi,high_tension
0,0,50.0,0,110,80,1,1,0,0,22.0,0
1,1,55.0,1,140,90,3,1,0,1,35.0,1
2,2,52.0,1,130,70,3,1,0,1,24.0,0
3,3,48.0,0,150,100,1,1,0,1,29.0,1
4,4,48.0,1,100,60,1,1,0,0,23.0,0


In [74]:
cardio_data["hypertension"]= (cardio_data["high_tension"]== 1)
cardio_data["hypertension"]=cardio_data["hypertension"].map({True:1,False:0}) 
cardio_data.head()

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi,high_tension,hypertension
0,0,50.0,0,110,80,1,1,0,0,22.0,0,0
1,1,55.0,1,140,90,3,1,0,1,35.0,1,1
2,2,52.0,1,130,70,3,1,0,1,24.0,0,0
3,3,48.0,0,150,100,1,1,0,1,29.0,1,1
4,4,48.0,1,100,60,1,1,0,0,23.0,0,0


#### 4) Las variables "hypertension" y "cholesterol" (cardio_data) son muy determinantes a la hora de desarrollar una "heart_disease" (variable considerada en train_data). Si ambos valores son altos, se aumenta considerablemente la posibilidad de sufrir una cardiopatía.

In [75]:
#cardio_data=cardio_data.drop(['ap_hi', 'ap_lo','cholesterol', 'high_tension'], axis=1) 
cardio_data["heart_disease"]=(cardio_data["hypertension"]==1) & (cardio_data["cholesterol"]>=2)
cardio_data["heart_disease"]=cardio_data["heart_disease"].map({True:1,False:0}) 
cardio_data.head()

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi,high_tension,hypertension,heart_disease
0,0,50.0,0,110,80,1,1,0,0,22.0,0,0,0
1,1,55.0,1,140,90,3,1,0,1,35.0,1,1,1
2,2,52.0,1,130,70,3,1,0,1,24.0,0,0,0
3,3,48.0,0,150,100,1,1,0,1,29.0,1,1,0
4,4,48.0,1,100,60,1,1,0,0,23.0,0,0,0


In [76]:
train_data.head()

,id,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,161.28,19.1,NaN,0


Las variables "bmi" y "smoking_status" (train_data) contienen valores nulos

In [77]:
train_data.isnull().any()

id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
avg_glucose_level    False
bmi                   True
smoking_status        True
stroke               False
dtype: bool

In [78]:
train_data.isnull().sum()

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

#### 5) "age" (train_data): pasamos el valor de días a años

In [79]:
train_data["age"]=round(train_data["age"],0)
train_data.head()

,id,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,161.28,19.1,NaN,0


#### 6) Transformaciones en los registros de los menores de edad, ya que se propone realizar distintos entrenamientos
En los valores de los niños menores de 10 años:
- si "smoking_status" es nulo, supondremos que no han fumado nunca
- si "bmi" es nulo, sustituimos el valor por la media de los que sí conocemos

In [80]:
#Los niños menores de 10 estamos asumiendo que no fuman 
child_data_10=train_data.copy()
child_data_10=child_data_10[(child_data_10["age"]<10)&(child_data_10["smoking_status"].isnull())]
child_data_10["smoking_status"]="never smoked"
#Calculamos la media de el bmi y se los sustituimos a los niños que tienen bmi NaN
child_data_10["bmi"]=child_data_10["bmi"].fillna(child_data_10["bmi"].mean())

En los valores de los adolescentes de 11 a 17 años:
- si "smoking_status" es nulo, se aplicará la proporción de los que sí conocemos como se indica en la siguiente celda
- si "bmi" es nulo, sustituimos el valor por la media de los que sí conocemos

In [81]:
#Adolecentes que no se saben si fuman , le asignamos segun la proporcion de los que si sabemos 
child_data_11=train_data.copy()
child_data_11=child_data_11[(child_data_11["age"]>=10)&(child_data_11["age"]<=17)&(child_data_11["smoking_status"].isnull())]
child_data_11["id"]=range(len(child_data_11))

#Elegimos el 81,5% no fuman 
child_data_11.smoking_status[0:1495]="never smoked"

#Elegimos el 13,5% dejaron de fumar 
child_data_11.smoking_status[1495:1743]="formerly smoked"

#Elegimos el 5% fuman 
child_data_11.smoking_status[1743:1834]="smokes"

#Calculamos la media de el bmi y se los sustituimos a los niños que tienen bmi NaN
child_data_11["bmi"]=child_data_11["bmi"].fillna(child_data_11["bmi"].mean())

C:\Users\sargonza\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\sargonza\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sargonza\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  de

In [82]:
#Unimos los datasets que toman en cuentas a los menores de edad 
child= pd.concat([child_data_11, child_data_10])
len(child)

6118

In [84]:
#Obtenemos los datos que no tienen smoking_status a NaN 
clean_data=train_data.copy()
clean_data=clean_data[clean_data["smoking_status"].notnull()]
print(len(clean_data), len(child))
clean_data=pd.concat([clean_data, child])


30108 6118


#### 7) Pasamos "smoking_status" (train_data) a los valores 0 ( never_smoked) y 1 (smokes, formerly_smoked)

In [85]:
#Convertir el smoking status a 0's y 1's
clean_data["smoking_status"]=clean_data["smoking_status"].map({"smokes":1, "formerly smoked":1, "never smoked":0})
clean_data.head()

,id,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
1,30468,Male,58.0,1,0,87.96,39.2,0,0
3,56543,Female,70.0,0,0,69.04,35.9,1,0
6,52800,Female,52.0,0,0,77.59,17.7,1,0
7,41413,Female,75.0,0,1,243.53,27.0,0,0
8,15266,Female,32.0,0,0,77.67,32.3,1,0


#### 8) "gender" (train_data) => 0 (hombre), 1 (mujer). Eliminamos los registros distintos a estos.

In [86]:
#Cambiamos de String a Number el gender y eliminamos los Others porque eran muy pocos 
clean_data["gender"]= clean_data["gender"].map({"Male":0, "Female":1, "Other":2})
clean_data= clean_data[clean_data["gender"]!=2]

#### 9) Eliminación de las columnas de cardio_data y train_data que no necesitamos

In [87]:

clean_data=clean_data.drop(['id'], axis=1) 
clean_data.head()

,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
1,0,58.0,1,0,87.96,39.2,0,0
3,1,70.0,0,0,69.04,35.9,1,0
6,1,52.0,0,0,77.59,17.7,1,0
7,1,75.0,0,1,243.53,27.0,0,0
8,1,32.0,0,0,77.67,32.3,1,0


In [88]:
cardio_data.head()

,id,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,cardio,bmi,high_tension,hypertension,heart_disease
0,0,50.0,0,110,80,1,1,0,0,22.0,0,0,0
1,1,55.0,1,140,90,3,1,0,1,35.0,1,1,1
2,2,52.0,1,130,70,3,1,0,1,24.0,0,0,0
3,3,48.0,0,150,100,1,1,0,1,29.0,1,1,0
4,4,48.0,1,100,60,1,1,0,0,23.0,0,0,0


In [89]:
cardio_data=cardio_data.drop(['id', 'ap_hi', 'ap_lo', 'cholesterol', 'high_tension'], axis=1) 
cardio_data.head() 

,age,gender,gluc,smoke,cardio,bmi,hypertension,heart_disease
0,50.0,0,1,0,0,22.0,0,0
1,55.0,1,1,0,1,35.0,1,1
2,52.0,1,1,0,1,24.0,0,0
3,48.0,0,1,0,1,29.0,1,0
4,48.0,1,1,0,0,23.0,0,0


#### 10) "gluc" (cardio_data) => pasamos los rangos a 0 (valor de la glucosa en sangre dentro de los valores normales) y 1(valores altos)
#### 11) "avg_glucose_level" (train_data) >=150 => "gluc" (train_data) =1 <=> "gluc" (cardio_data) =1

In [90]:
cardio_data["gluc"]= cardio_data["gluc"].map({1:0, 2:1, 3:1})
cardio_data.head()

,age,gender,gluc,smoke,cardio,bmi,hypertension,heart_disease
0,50.0,0,0,0,0,22.0,0,0
1,55.0,1,0,0,1,35.0,1,1
2,52.0,1,0,0,1,24.0,0,0
3,48.0,0,0,0,1,29.0,1,0
4,48.0,1,0,0,0,23.0,0,0


In [91]:
clean_data["gluc"]=clean_data["avg_glucose_level"]>=150
clean_data["gluc"]=clean_data["gluc"].map({False:0, True:1})

clean_data.head()

,gender,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke,gluc
1,0,58.0,1,0,87.96,39.2,0,0,0
3,1,70.0,0,0,69.04,35.9,1,0,0
6,1,52.0,0,0,77.59,17.7,1,0,0
7,1,75.0,0,1,243.53,27.0,0,0,1
8,1,32.0,0,0,77.67,32.3,1,0,0


In [92]:
clean_data=clean_data.drop(['avg_glucose_level'], axis=1) 
clean_data.head() 

,gender,age,hypertension,heart_disease,bmi,smoking_status,stroke,gluc
1,0,58.0,1,0,39.2,0,0,0
3,1,70.0,0,0,35.9,1,0,0
6,1,52.0,0,0,17.7,1,0,0
7,1,75.0,0,1,27.0,0,0,1
8,1,32.0,0,0,32.3,1,0,0


#### 12) Renombramos las columnas para que tengan el mismo nombre en ambos datasets

In [93]:
#clean_data.rename(index={0: "gender", 1: "age", 2: "hypertension", 3: "heart_disease", 4: "bmi",  5: "smoke", 6: "cardio", 7: "gluc"})
#clean_data.rename(columns={"smoking_status":"smoke", "stroke":"cardio"})

clean_data["smoke"]=clean_data["smoking_status"]
clean_data["cardio"]=clean_data["stroke"]

In [94]:
clean_data=clean_data.drop(['smoking_status', 'stroke'], axis=1) 
clean_data.head() 

,gender,age,hypertension,heart_disease,bmi,gluc,smoke,cardio
1,0,58.0,1,0,39.2,0,0,0
3,1,70.0,0,0,35.9,0,1,0
6,1,52.0,0,0,17.7,0,1,0
7,1,75.0,0,1,27.0,1,0,0
8,1,32.0,0,0,32.3,0,1,0


In [95]:
cardio_data.head()

,age,gender,gluc,smoke,cardio,bmi,hypertension,heart_disease
0,50.0,0,0,0,0,22.0,0,0
1,55.0,1,0,0,1,35.0,1,1
2,52.0,1,0,0,1,24.0,0,0
3,48.0,0,0,0,1,29.0,1,0
4,48.0,1,0,0,0,23.0,0,0


### Unimos los dos datasets, creando uno único con un total de 106217 registros

In [179]:
#print(len(clean_data), len(cardio_data))
union = pd.concat([cardio_data, clean_data])
union.head()

C:\Users\sargonza\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,age,bmi,cardio,gender,gluc,heart_disease,hypertension,smoke
0,50.0,22.0,0,0,0,0,0,0
1,55.0,35.0,1,1,0,1,1,0
2,52.0,24.0,1,1,0,0,0,0
3,48.0,29.0,1,0,0,0,1,0
4,48.0,23.0,0,1,0,0,0,0


In [180]:
union["bmi"]=union["bmi"].fillna(union["bmi"].mean())
union["bmi"].isnull().any()
len(union)

106217

Agrupamos por rangos el "bmi" para poder distinguirlo en:

In [181]:
# 0: Inferior al normal  1:Normal  2:Sobrepeso 3:Obesidad 
def bmi_clf(data):
    if data <18.5: return 0
    elif 18.5>=data<24.9: return 1
    elif 24.9>=data<29.9: return 2
    else:return 3

In [182]:
union['bmi'] = list(map(bmi_clf, union['bmi']))
union.head()

,age,bmi,cardio,gender,gluc,heart_disease,hypertension,smoke
0,50.0,2,0,0,0,0,0,0
1,55.0,3,1,1,0,1,1,0
2,52.0,2,1,1,0,0,0,0
3,48.0,3,1,0,0,0,1,0
4,48.0,2,0,1,0,0,0,0


Hacemos lo mismo para "age"

In [183]:
def age_clf(data):
    if 30>=data<50: return 0
    elif 50>=data<70: return 1
    elif 70>=data<90: return 2
    else:return 3

In [184]:
union['age'] = list(map(age_clf, union['age']))
union=union[union["age"]!=-3]


Casos de "cardio" == 1 por edades:
- 0-19 : 2
- 20-49 : 7930
- 50-69 : 27337
- 70-85 : 350

In [185]:
len(union)

106217

In [186]:
# Del total de 106217 registros tenemos 35619 que sufren un infarto.
union[union["cardio"]==1].sum()

age              62599
bmi              97547
cardio           35619
gender           22978
gluc              6617
heart_disease     6338
hypertension     15995
smoke             3283
dtype: int64

Guardamos en un nuevo archivo csv el dataset único que hemos creado:
- all.csv => todos los registros
- adult_data => los registros de los mayores de edad

In [187]:
print("Comprobacioón Datos:\n \n Edad:",union["age"].unique(),\
      "\n \n BMI:",union["bmi"].unique()," Cardio:",union["cardio"].unique(),\
      " Gender:",union["gender"].unique()," Glucose:",union["gluc"].unique(),\
      " Heart Disease:",union["heart_disease"].unique(),\
      " Hypertension:",union["hypertension"].unique(),
     " Smoke:",union["smoke"].unique(),)

Comprobacioón Datos:
 
 Edad: [1 2 0 3] 
 
 BMI: [2 3 0 1]  Cardio: [0 1]  Gender: [0 1]  Glucose: [0 1]  Heart Disease: [0 1]  Hypertension: [0 1]  Smoke: [0 1]


In [188]:
all_union=union.copy()
#all_union=all_union[all_union["age"]>=18]
all_union.to_csv('people_II.csv',index=False, header=True) 

In [189]:
adult_union=union[union["age"]>=18]
adult_union.to_csv('adult_data_II.csv',index=False, header=True) 